# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd
import time

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')

In [3]:
driver.find_element_by_id('mcrdata').send_keys('006179570C')


In [4]:
driver.find_element_by_id('submit3').click()

In [5]:

name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text

owner= driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text

phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text

license_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]').text

pysical_address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split(" ")[-7:-1]

mylist= [name,owner,phone,license_status,pysical_address]
mylist

['Name:   B.D. SMITH TOWING',
 'Owner/Officer:   BRANDT SMITH / OWNER',
 'Phone:   8173330706',
 'Status:  Expired',
 ['76179\n\nPhysical:\n13619',
  'BRETT',
  'JACKSON',
  'RD.\nFORT',
  'WORTH,',
  'TX.']]

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [6]:

name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text

owner= driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text

phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text

license_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]').text

pysical_address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split(" ")[-7:-1]

print(name)
print(owner)
print(phone)
print(license_status)
print(pysical_address)

Name:   B.D. SMITH TOWING
Owner/Officer:   BRANDT SMITH / OWNER
Phone:   8173330706
Status:  Expired
['76179\n\nPhysical:\n13619', 'BRETT', 'JACKSON', 'RD.\nFORT', 'WORTH,', 'TX.']


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [7]:
df=pd.read_csv('trucks-subset.csv')
df.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [8]:
df

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


In [9]:
# method 1 
def get_url( row):
    
    if row == '006507931C':
        return "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row
    if row == '006179570C':
        return "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" +row
    if row == '006502097C':
        return "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row
    
get_url('006507931C')
      

'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C'

In [10]:
# better way!

# def get_url(row):
#     url ="https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber="+row['TDLR Number']
    

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [11]:
df['url'] = df['TDLR Number'].apply(get_url)
df.head()

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_d...
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_d...
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_d...


## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [12]:
def get_info(row):
    # get each url from the df
    driver.get(row['url'])
    
    # grab info
    name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    owner= driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    license_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]').text
    pysical_address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split(" ")[-7:-1]
    
    
    print(name)
    print(owner)
    print(phone)
    print(license_status)
    print(pysical_address)
    
    return pd.Series({
        'name':name,
        'owner/oprater':owner,
        'phone':phone,
        "license status":license_status,
        "pysical address":pysical_address
    })

# in order to print them out! we need use .apply

In [13]:
df2 = df.apply(get_info, axis=1)
df2

Name:   AUGUSTUS E SMITH
Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER
Phone:   9032276464
Status:  Active
['TX.', '75418\n\nPhysical:\n103', 'N', 'MAIN', 'ST\nBONHAM,', 'TX.']
Name:   AUGUSTUS E SMITH
Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER
Phone:   9032276464
Status:  Active
['TX.', '75418\n\nPhysical:\n103', 'N', 'MAIN', 'ST\nBONHAM,', 'TX.']
Name:   B.D. SMITH TOWING
Owner/Officer:   BRANDT SMITH / OWNER
Phone:   8173330706
Status:  Expired
['76179\n\nPhysical:\n13619', 'BRETT', 'JACKSON', 'RD.\nFORT', 'WORTH,', 'TX.']
Name:   BARRY MICHAEL SMITH
Owner/Officer:   BARRY MICHAEL SMITH / OWNER
Phone:   8066544404
Status:  Active
['TX.', '79015\n\nPhysical:\n4501', 'W', 'CEMETERY', 'RD\nCANYON,', 'TX.']


,name,owner/oprater,phone,license status,pysical address
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"[TX., 75418\n\nPhysical:\n103, N, MAIN, ST\nBO..."
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"[76179\n\nPhysical:\n13619, BRETT, JACKSON, RD..."
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"[TX., 79015\n\nPhysical:\n4501, W, CEMETERY, R..."


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [14]:
df2.to_csv('tow-trucks-extended.csv',index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [15]:
df2 = pd.read_csv('tow-trucks-extended.csv')
df2.head()

,name,owner/oprater,phone,license status,pysical address
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"['TX.', '75418\n\nPhysical:\n103', 'N', 'MAIN'..."
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"['76179\n\nPhysical:\n13619', 'BRETT', 'JACKSO..."
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"['TX.', '79015\n\nPhysical:\n4501', 'W', 'CEME..."


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [22]:
df3= pd.read_csv('tow-trucks.csv')
df3.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C
3,006494912C
4,0649468VSF


In [23]:
# get url row
def get_all_url( row):
    
    url = "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber="+ row['TDLR Number']
    
    return url
    

In [24]:
df3['url']=df3.apply(get_all_url,axis=1)
df3

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_d...
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_d...
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_d...
3,006494912C,https://www.tdlr.texas.gov/tools_search/mccs_d...
4,0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_d...
5,006448786C,https://www.tdlr.texas.gov/tools_search/mccs_d...
6,0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_d...
7,0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_d...
8,006017767C,https://www.tdlr.texas.gov/tools_search/mccs_d...
9,006495492C,https://www.tdlr.texas.gov/tools_search/mccs_d...


In [25]:
def get_all_info(row):
    # get each url from the df
    driver.get(row['url'])
    
    # grab info
    name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    owner= driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    license_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]').text
    pysical_address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split(" ")[-7:-1]
    
    
    print(name)
    print(owner)
    print(phone)
    print(license_status)
    print(pysical_address)
    
    return pd.Series({
        'name':name,
        'owner/oprater':owner,
        'phone':phone,
        "license status":license_status,
        "pysical address":pysical_address
    })

new_df=df3.apply(get_all_info,axis=1).join(df3)




Name:   AUGUSTUS E SMITH
Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER
Phone:   9032276464
Status:  Active
['TX.', '75418\n\nPhysical:\n103', 'N', 'MAIN', 'ST\nBONHAM,', 'TX.']
Name:   AUGUSTUS E SMITH
Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER
Phone:   9032276464
Status:  Active
['TX.', '75418\n\nPhysical:\n103', 'N', 'MAIN', 'ST\nBONHAM,', 'TX.']
Name:   B.D. SMITH TOWING
Owner/Officer:   BRANDT SMITH / OWNER
Phone:   8173330706
Status:  Expired
['76179\n\nPhysical:\n13619', 'BRETT', 'JACKSON', 'RD.\nFORT', 'WORTH,', 'TX.']
Name:   BARRY MICHAEL SMITH
Owner/Officer:   BARRY MICHAEL SMITH / OWNER
Phone:   8066544404
Status:  Active
['TX.', '79015\n\nPhysical:\n4501', 'W', 'CEMETERY', 'RD\nCANYON,', 'TX.']
Name:   HEATH SMITH
Owner/Officer:   HEATH A SMITH / OWNER
Phone:   940-552-0687
Status:  Expired
['ST\nVERNON,', 'TX.', '76384\n\nPhysical:\n1529', 'WILBARGER', 'ST\nVERNON,', 'TX.']
Name:   HEATH SMITH
Owner/Officer:   HEATH A SMITH / OWNER
Phone:   9405520687
Status:  Expired

In [26]:
new_df.head()

,name,owner/oprater,phone,license status,pysical address,TDLR Number,url
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"[TX., 75418\n\nPhysical:\n103, N, MAIN, ST\nBO...",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_d...
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"[76179\n\nPhysical:\n13619, BRETT, JACKSON, RD...",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_d...
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"[TX., 79015\n\nPhysical:\n4501, W, CEMETERY, R...",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_d...
3,Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 940-552-0687,Status: Expired,"[ST\nVERNON,, TX., 76384\n\nPhysical:\n1529, W...",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_d...
4,Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 9405520687,Status: Expired,"[ST\nVERNON,, TX., 76384\n\nPhysical:\n1529, W...",0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_d...


In [27]:
new_df.to_csv('tow-trucks.csv',index=False)